In [ ]:
import numpy as np
import pandas as pd
import math
import random
from sklearn.preprocessing import normalize
from sklearn.impute import SimpleImputer
from numpy import linalg as LA

In [ ]:
# add data
dataset = pd.read_csv('CC_GENERAL.csv')
X = dataset.iloc[:,1:]
# print(X)

In [ ]:
# handling missing values
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X)
X = imputer.transform(X)
X = pd.DataFrame(X)
# print(X)

In [ ]:
# normalizing according to columns
X = normalize(X, axis=1)
# print(X)

In [ ]:
# distance between a centre and each point
def distance(centers, point):
    min_dist = math.inf
    for c in range(len(centers)):
        curr_dist = LA.norm((np.array(np.subtract(centers[c], point))))
        if curr_dist < min_dist:
            min_dist = curr_dist
#         print(curr_dist)
    return min_dist
        

In [ ]:
# function to select k centers
def greedy_k(data, p):
    ans = []
    c = random.randrange(len(data))
    # print(c)
    ans.append(data[c])
    np.delete(data, c)
    num = 1
    while(num<p):
        max_dis = 0
        new_center = 0
        for row in range(len(data)):
            dist = distance(ans, data[row])
            if dist>max_dis:
                max_dis = dist
                new_center = row
        ans.append(data[new_center])
        np.delete(data, new_center)
        num+=1
    return ans, data

In [ ]:
# greedy k center algorithm
num_ctrs = [2,4,10]
greedy_costs = []
for k in num_ctrs:
    objective = 0
    M = X[0:50]
    centers, mod_data = greedy_k(M, k)
    for row in mod_data:
        min_d = math.inf
        for ct in centers:
            currd = LA.norm(np.array(np.subtract(ct, row)))
            if currd < min_d:
                min_d = currd
        objective = max(objective, min_d)
    greedy_costs.append(objective)
for j in range(len(num_ctrs)):
  print("k = ", num_ctrs[j], "cost of clustering = ", greedy_costs[j])

k =  2 cost of clustering =  1.0334155972714831
k =  4 cost of clustering =  0.7877457047941936
k =  10 cost of clustering =  0.48459575749097517


In [ ]:
# optimal k-center algorithm
from itertools import combinations
def optimal_k(data, k):
    arr = [i for i in range(len(data))]
    # print(arr)
    centers = list(combinations(arr, k))
    final_cost = math.inf
    for curr_centers in centers:
        objective = 0
        for row in range(len(data)):
            if row not in curr_centers:
                min_d = math.inf
                for center in curr_centers:
                    currd = LA.norm(np.array(np.subtract(data[center], data[row])))
                    if currd < min_d:
                        min_d = currd
                objective = max(objective, min_d)
        if objective < final_cost:
            final_cost = objective
            final_center = curr_centers
    return final_center, final_cost

In [ ]:
num_ctrs = [2, 4]
opt_costs = []
for k in num_ctrs:
    M = X[0:50]
    cost = optimal_k(M, k)
    opt_costs.append(cost[1])
    print("k = ", k, "cost of clustering = ", cost[1])

k =  2 cost of clustering =  0.8312290826212347
k =  4 cost of clustering =  0.6171438786897351


In [ ]:
# approximation factor
print("For k = 2, approximation factor is ")
print(greedy_costs[0]/opt_costs[0])
print()
print("For k = 4, approximation factor is ")
print(greedy_costs[1]/opt_costs[1])

For k = 2, approximation factor is 
1.2432380181076732

For k = 4, approximation factor is 
1.2764376865677824
